# Introducción: Optimizar el retorno en CTAs

El objetivo de esta notebook es hacer una POC para Optimizar el conjunto de individuos a contactar para todos las CTA activos.
Es decir, definir el tensor Cliente, CTA, Canal con dos posibles valores:
*   1: Activo 
*   0: No activo

Para lograr la solución final, debemos optimizar una ecuación de valor que depende de la rentabilidad, la probabilidad de éxito y del costo:
*   $Valor = (Rentabilidad$ $del$ $producto)$ x $(Probabilidad$ $exito$ $del$ $conjunto$ $de$ $canales$ $seleccionado) - (Costo$ $de$ $la$ $acción)$

De la siguiente ecuación el costo es algo que se estima con promedios, y es algo relativamente conocido. El resto de las partes de la ecuación las debemos estimar. Por ende, la solución final se plantea de la siguiente manera: 

1.   Hallar las probabilidades de éxito para el tensor cliente, CTA, canales utilizados usando una regresión logística, así como también la estimación del aumento en el valor de cliente asociado a aceptar el producto (rentabilidad mensual por esperanza de vida). Para realizar esto se deben utilizar datos "históricos".

2.   Usar un algoritmo de optimización discreta con restricciones para optimizar la ecuación de valor.

Se utilizarán datos simulados, para intentar alcanzar un escenario real. Por otro lado, cuando el usuario va ejecutando la notebook, la misma lo va orientando mediante preguntas, las cuales sirven para la configuración de las restricciones.




# Modelado de la rentabilidad y la probabilidad de éxito

### importación de librerias 

In [ ]:
# Importa librerias
!pip install pulp
!pip install Faker
from pulp import *
import numpy as np
import pandas as pd
from faker import Faker
import re
from dateutil.relativedelta import relativedelta
import datetime as dt
import os
import pickle
from sklearn import metrics

from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.model_selection import train_test_split

import warnings
warnings.filterwarnings("ignore")
from sklearn.linear_model import LogisticRegression

## Generación de datos

### Número de clientes a modelar
Estos son los clientes que quiero utilizar para modelar la probabilidad de éxito y el aumento en la rentabilidad asociado al éxito de una campaña

In [ ]:
definir=int(input('Desea definir el número de clientes a considerar en el modelo (1: Si, 0: No): '))
if definir==1:
  N=int(input('Defina el número de clientes a modelar: ')) #10000
elif definir==0:
  N=10000
else:
  raise RuntimeError('Error: Solamente puede escribir 1 o 0, otros valores no son permitidos') # manejo de errores

Desea definir el número de clientes a considerar en el modelo (1: Si, 0: No): 0


### Establezco los productos a ofrecer y los canales

Los productos (simil campañas) a utilizar son:
*   Seguros de automovil
*   Prestamo

Se van a utilizar 3 **canales**, por lo cual vamos a tener 8 combinaciones de canales (3 de a pares + 3 únicos + todos + ninguno). Los canales son:
* SMS: se enviarán promedio dos SMS por persona, en caso de enviar.
* Mail: Se enviarán un promedio de 4 mail por persona.
* Call center: se hará un promedio de 2 llamadas de 5 minutos cada una.

In [ ]:
Productos = ['SEGURO_AUTOMOVIL','PRESTAMO']
Productos_Res = ['SA','PR']

lista_canales = ['ALL', 'CC_MAIL', 'CC_SMS', 'MAIL_SMS', 'CC', 'MAIL', 'SMS', 'NONE']

### Simular probabilidad de éxito.

Se debe simular la probabilidad de éxito en cada uno de los canales, para cada uno de los productos. Esto sirve para simular posteriormente el éxito en cada uno de los datos (distribución bernoulli):

In [ ]:
Probabilidad={}
Probabilidad.update({'NONE_SA':np.random.uniform(0, 0.001, size=None),
                      'NONE_PR':np.random.uniform(0, 0.001, size=None)})
  
Probabilidad.update({'CC_SA': np.random.uniform(Probabilidad['NONE_SA'], 0.1, size=None), 
                    'MAIL_SA':np.random.uniform(Probabilidad['NONE_SA'] , 0.01, size=None),  
                    'SMS_SA':np.random.uniform(Probabilidad['NONE_SA'] , 0.025, size=None), 
                    'CC_PR':np.random.uniform(Probabilidad['NONE_PR'], 0.1, size=None),
                    'MAIL_PR':np.random.uniform(Probabilidad['NONE_PR'] , 0.01, size=None),
                    'SMS_PR':np.random.uniform(Probabilidad['NONE_PR'] , 0.025, size=None)})

Probabilidad.update({'CC_MAIL_SA':np.random.uniform(min([Probabilidad['MAIL_SA'],Probabilidad['CC_SA']]), 0.110, size=None),
                      'CC_SMS_SA': np.random.uniform(min([Probabilidad['SMS_SA'],Probabilidad['CC_SA']]), 0.125, size=None), 
                      'MAIL_SMS_SA':np.random.uniform(min([Probabilidad['MAIL_SA'],Probabilidad['SMS_SA']]), 0.035, size=None), 
                      'CC_MAIL_PR':np.random.uniform(min([Probabilidad['MAIL_PR'],Probabilidad['CC_PR']]), 0.110, size=None),
                      'CC_SMS_PR': np.random.uniform(min([Probabilidad['SMS_PR'],Probabilidad['CC_PR']]), 0.125, size=None), 
                      'MAIL_SMS_PR':np.random.uniform(min([Probabilidad['MAIL_PR'],Probabilidad['SMS_PR']]), 0.035, size=None)})

Probabilidad.update({'ALL_SA':np.random.uniform(min([Probabilidad['CC_MAIL_SA'],Probabilidad['CC_SMS_SA'],Probabilidad['MAIL_SMS_SA']]), 0.135, size=None),
                     'ALL_PR':np.random.uniform(min([Probabilidad['CC_MAIL_PR'],Probabilidad['CC_SMS_PR'],Probabilidad['MAIL_SMS_PR']]), 0.135, size=None)})

### Tablas a utilizar para el modelo

En este paso se crea las tablas utilizadas para la generación del modelo de probabilidad de éxito y de rentabilidad. Las mismas representan datos de:

* Cadastro de clientes
* Rentabilidad
* Campañas realizadas


In [ ]:
#datos catastrales de clientes
Cadastro = []
faker = Faker('es_ES')
for n in range(N):
    Cadastro.append(list(faker.profile().values()))
Cadastro = pd.DataFrame(Cadastro, columns=faker.profile().keys())
Cadastro['ContaID'] = range(N)


#datos de rentabilidad
Dados_Receita = pd.DataFrame(10 * np.random.rand(3* N,6), columns=['VlCompraNacional', 'VlCompraInterNacional', 'VlBilheteTransporte', 'VlCargaCelular','VlTarifaManutençao', 'VlOutrasTarifas'])
Dados_Receita['Rentabilidade'] = 0.0124 * Dados_Receita['VlCompraNacional'] + 0.0148 * Dados_Receita['VlCompraInterNacional'] + 0.02 * Dados_Receita['VlBilheteTransporte'] + 0.02 * Dados_Receita['VlCargaCelular'] + Dados_Receita['VlTarifaManutençao'] + Dados_Receita['VlOutrasTarifas']
Dados_Receita['Anomes'] = 1
Dados_Receita['ContaID'] = [*range(N), *range(N), *range(N)]
Dados_Receita['Anomes'][Dados_Receita.index >= N] = 2
Dados_Receita['Anomes'][Dados_Receita.index >= 2*N] = 3

#La rentabilidad como es a futuro y no por mes, se multiplicará por la esperanza de vida a futuro (se pone por 12) 
Dados_Receita['Rentabilidade'] = Dados_Receita['Rentabilidade'] * 12

# Define tabla de datos de campañas
Dados_Campanhas = pd.DataFrame(columns=['Campanha', 'Canal', 'Exito'])

#general aleatoriamente los datos de producto y canal asociado
Dados_Campanhas['Campanha'] = np.random.choice(Productos, size=2*N)
Dados_Campanhas['Canal'] = np.random.choice(lista_canales, size=2*N)


#define el éxito de la campaña, con una probabilidad 0.1
Dados_Campanhas['Exito'] = 0 #np.random.binomial(1, 0.1, 2*N)
for Campanha in Productos:
  for Canal in lista_canales:
    total=len(Dados_Campanhas[(Dados_Campanhas.Canal == Canal) & (Dados_Campanhas.Campanha == Campanha)])
    Dados_Campanhas.loc[(Dados_Campanhas.Canal == Canal) & 
                        (Dados_Campanhas.Campanha == Campanha),
                        'Exito']= np.random.binomial(1,Probabilidad[Canal+'_'+Productos_Res[Productos.index(Campanha)]],total)


Dados_Campanhas['Anomes'] = 1
Dados_Campanhas['ContaID'] = range(2*N)
Dados_Campanhas['Anomes'][Dados_Campanhas.index >= N] = 2
Dados_Campanhas['ContaID'][Dados_Campanhas.index >= N] = range(N)


#calculo de rentabilidad asociada
Rec_Camp_df = Dados_Receita.merge(Dados_Campanhas, how='left', left_on=['Anomes', 'ContaID'], right_on=['Anomes', 'ContaID'])
Rec_Camp_df['AumentoRent'] = 0
#Rec_Camp_df['AumentoRent'] = Rec_Camp_df['RentabilidadeM2] - Rec_Camp_M1_df['RentabilidadeM1']
for anomes in range(1, 3):
  a = Rec_Camp_df.loc[(Rec_Camp_df.Anomes == anomes) | (Rec_Camp_df.Anomes == anomes + 1)]
  #a.loc[a.Anomes == anomes, 'Rentabilidade'] = a.Rentabilidade
  a = a.groupby('ContaID',  as_index=False).sum()
  a['ContaID'] = a['ContaID'] + N * (anomes - 1)
  a = a.set_index('ContaID')
  a = a['Rentabilidade']
  Rec_Camp_df.loc[Rec_Camp_df.Anomes == anomes, 'AumentoRent'] = round(a*Rec_Camp_df.loc[Rec_Camp_df.Anomes == anomes, 'Exito'] ,3)
  
Rec_Camp_df = Rec_Camp_df[['ContaID', 'Anomes', 'Campanha', 'Canal', 'Exito', 'AumentoRent']]
Rec_Camp_df.head()

,ContaID,Anomes,Campanha,Canal,Exito,AumentoRent
0,0,1,SEGURO_AUTOMOVIL,MAIL,0.0,0.0
1,1,1,SEGURO_AUTOMOVIL,CC_MAIL,0.0,0.0
2,2,1,SEGURO_AUTOMOVIL,MAIL,0.0,0.0
3,3,1,SEGURO_AUTOMOVIL,CC,0.0,0.0
4,4,1,PRESTAMO,CC_MAIL,0.0,0.0


En este paso lo que se hace es unir todas las tablas con el objetivo de  realizar el modelo para la predicción de la rentabilidad. además de crear algunas variables dummies.

In [ ]:
Cadastro_Modelo = Cadastro[['ContaID', 'job', 'sex', 'birthdate']]

# job
Mais_comuns = pd.value_counts(Cadastro_Modelo['job']).head(10)
Cadastro_Modelo.loc[~Cadastro_Modelo['job'].isin(Mais_comuns.index), 'job'] = 'Outros'
Cadastro_Modelo = pd.get_dummies(Cadastro_Modelo, columns = ['job'])

#sex
Cadastro_Modelo = pd.get_dummies(Cadastro_Modelo, columns = ['sex'])

#Idade
ref_date = dt.datetime.now()
Cadastro_Modelo['age'] = Cadastro_Modelo['birthdate'].apply(lambda x: len(pd.date_range(start = x, end = ref_date, freq = 'Y'))) 
Cadastro_Modelo = Cadastro_Modelo.drop(['birthdate'], axis = 1)

Receita_Modelo = Dados_Receita.groupby(['ContaID']).mean()
Receita_Modelo = Receita_Modelo.drop(['Anomes', 'Rentabilidade'], axis = 1)


modelo_df = Rec_Camp_df.merge(Cadastro_Modelo, how='left', left_on='ContaID', right_on='ContaID')
modelo_df = modelo_df.merge(Receita_Modelo, how='left', left_on='ContaID', right_on='ContaID')


modelo_df_com_dados = modelo_df[modelo_df.Exito == 1]
modelo_df_com_dados = modelo_df_com_dados.drop(['Campanha', 'Canal', 'Anomes', 'ContaID', 'Exito'], axis=1)
modelo_df_com_dados.head()

,AumentoRent,job_Bonds trader,job_Building services engineer,job_Chief Strategy Officer,"job_Engineer, chemical",job_Forensic psychologist,"job_Geneticist, molecular",job_Occupational hygienist,job_Outros,job_Psychotherapist,job_Records manager,"job_Scientist, marine",sex_F,sex_M,age,VlCompraNacional,VlCompraInterNacional,VlBilheteTransporte,VlCargaCelular,VlTarifaManutençao,VlOutrasTarifas
26,135.536,0,0,0,0,0,0,0,1,0,0,0,1,0,2,4.925805,4.090259,5.594859,5.433773,3.212092,3.102432
30,329.866,0,0,0,0,0,0,0,1,0,0,0,1,0,74,1.675244,4.513975,4.816747,6.698114,4.312768,6.597538
77,164.795,0,0,0,0,0,0,0,1,0,0,0,1,0,105,4.732032,7.195686,4.158930,3.540179,3.937121,5.273034
99,258.550,0,0,0,0,0,0,0,1,0,0,0,1,0,75,7.170591,5.221253,7.496320,3.589269,6.163006,3.863774
129,188.753,0,0,0,0,0,0,0,1,0,0,0,1,0,48,5.060494,5.446359,4.023727,5.509103,5.610083,4.474800


## Modelado

### Modelado de la rentabilidad

En este paso para cada uno de los productos se modela el aumento del valor de cliente aosiado al éxito de la campaña y se guarda en un archivo .pkl, este archivo se utilizará despues en la estimación de la rentabilidad


In [ ]:
for Campanha in Productos:
    modelo_rentabilidad = RandomForestRegressor(max_depth=10, random_state=0)
   

    modelo_df_com_dados = modelo_df[(modelo_df.Campanha == Campanha) & (modelo_df.Exito == 1)]
    modelo_df_com_dados = modelo_df_com_dados.drop(['Campanha', 'Anomes', 'ContaID', 'Exito','Canal',
                                                    'VlCompraNacional',	'VlCompraInterNacional',	'VlBilheteTransporte',
                                                    'VlCargaCelular',	'VlTarifaManutençao',	'VlOutrasTarifas'], axis=1) 
    #modelo_df_com_dados = pd.get_dummies(modelo_df_com_dados, columns = ['Canal'])
    
    X = modelo_df_com_dados.drop(['AumentoRent'], axis=1)
    y = modelo_df_com_dados['AumentoRent']

    X_train, X_test, y_train, y_test = train_test_split( X, y, test_size=0.1, random_state=42)
    modelo_rentabilidad.fit(X_train, y_train)
    file_name='Rentabilidad_'+Productos_Res[Productos.index(Campanha)]+'.pkl'
    with open(os.path.join('sample_data', file_name), 'wb') as f:
      pickle.dump(modelo_rentabilidad, f)

X1=X.head()

### Modelado de la probabilidad


In [ ]:
for Campanha in Productos:
  Probabilidad =  LogisticRegression(random_state=0)

  modelo_df_com_dados = modelo_df[modelo_df.Campanha == Campanha ]
  modelo_df_com_dados = modelo_df_com_dados.drop(['Campanha', 'Anomes', 'ContaID', 'AumentoRent',
                                                  'VlCompraNacional',	'VlCompraInterNacional',	'VlBilheteTransporte',
                                                  'VlCargaCelular',	'VlTarifaManutençao',	'VlOutrasTarifas'], axis=1)
  modelo_df_com_dados = pd.get_dummies(modelo_df_com_dados, columns = ['Canal'])

  X = modelo_df_com_dados.drop(['Exito'], axis=1)
  y = modelo_df_com_dados['Exito']

  X_train, X_test, y_train, y_test = train_test_split( X, y, test_size=0.2, random_state=42)
  Probabilidad.fit(X_train, y_train)

  file_name='Probabilidad_'+Productos_Res[Productos.index(Campanha)]+'.pkl'
  with open(os.path.join('sample_data', file_name), 'wb') as f:
    pickle.dump(Probabilidad, f)

X2=X.head()

# Optimización del retorno en cada CTA

Se mostrará como funcionaría el modelo de optimización para un universo muy acotado, pero que intenta simular un escenario real. El usuario final podrá elegir la cantidad de **clientes** en el banco y las restricciones utilizadas.

Tendremos **restricciones configurables** de:
* Costo total. El mismo no puede superar cierto umbral por presupesto de marketing.
* costo por cliente. 
* Cantidad de contactos previos por canal en los últimos meses: no queremos interactuar con el cliente mas de una cierta cantidad de veces.
* El producto como mucho se ofrecerá por un conjunto de canales.
* capacidad del call center no puede superar un cierto umbral

Por otro lado como **restricciones no configurables** de:
* el cliente no quiere ser contactado por algún canal.

Si bien ya lo mencionamos el problema se resuelve utilizando programación lineal entera ya que la decisión para cada elemento de la ecuación es binaria (le ofrezco al cliente el producto a por los canales i,j,k o no). La ecuación a maximizar es (cliente: i, producto: j, conjunto canal: k):

* $Probabilidad_{i,j,k}*Rentabilidad_{j}*Eleccion_{i,j,k}-Costo_{k}$

Sujeta a las restricciones mencionadas.


## Establecimiento de parámetros



### Establecimiento del número de clientes

In [ ]:
#Número de clientes
definir=int(input('Desea definir el número de clientes a considerar (1: Si, 0: No): '))
if definir==1:
  nro_clientes=int(input('Defina el número de clientes a considerar: ')) #100
elif definir==0:
  nro_clientes=100
else:
  raise RuntimeError('Error: Solamente puede escribir 1 o 0, otros valores no son permitidos') # manejo de errores

Desea definir el número de clientes a considerar (1: Si, 0: No): 0


In [ ]:
#Creo la lista de productos (Está creado más arriba)
Productos = ['SEGURO_AUTOMOVIL','PRESTAMO']
Productos_Res = ['SA','PR']

#Creo la lista de canales
Canales = ['CC','MAIL','SMS']

#Elecciones posibles
El = ['ALL_SA', 'CC_MAIL_SA', 'CC_SMS_SA', 'MAIL_SMS_SA', 'CC_SA', 'MAIL_SA', 'SMS_SA', 'NONE_SA',
            'ALL_PR', 'CC_MAIL_PR', 'CC_SMS_PR', 'MAIL_SMS_PR', 'CC_PR', 'MAIL_PR', 'SMS_PR', 'NONE_PR']


### Establecimiento de costos de cada uno de los canales de interacción 

In [ ]:
definir=int(input('Desea definir el costo de cada uno de los canales de interacción (1: Si, 0: No): '))
if definir==1:
  Costos=[]
  for i in range(0, len(Canales)): 
      ele = float(input('Defina el costo de cada '+Canales[i]+': ')) #25, 0.1, 1
      Costos.append(ele) # adding the element     

  Promedio_interacciones={'CC':2, 'MAIL':4, 'SMS':2} #por letra
  Costos_canal=dict(zip(Canales,[Promedio_interacciones['CC']*Costos[0], 
                                Promedio_interacciones['MAIL']*Costos[1], 
                                Promedio_interacciones['SMS']*Costos[2]])) 
elif definir==0:
  Costos_canal={'CC':25, 'MAIL':0.1, 'SMS':1}
else:
  raise RuntimeError('Error: Solamente puede escribir 1 o 0, otros valores no son permitidos') # manejo de errores

Desea definir el costo de cada uno de los canales de interacción (1: Si, 0: No): 0


### Restricciones de presupuestos

Aca el cliente va a definir los parámetros asociados al presupuesto de marketing y al costo que se quiere definir por cliente

In [ ]:
#restricción de costo (presupuesto Marketing)
definir=int(input('Desea definir el presupuesto total de marketing (1: Si, 0: No): '))
if definir==1:
  Costo_total=int(input('Defina el presupuesto total de marketing: ')) #10000
elif definir==0:
  Costo_total=10000
else:
  raise RuntimeError('Error: Solamente puede escribir 1 o 0, otros valores no son permitidos') # manejo de errores


#restricción de costo por cliente
definir=int(input('Desea definir el costo máximo que se desea asumir por cliente (1: Si, 0: No): '))
if definir==1:
  Costo_cliente=int(input('Defina el costo máximo que se desea asumir por cliente: ')) #100
elif definir==0:
  Costo_cliente=100
else:
  raise RuntimeError('Error: Solamente puede escribir 1 o 0, otros valores no son permitidos') # manejo de errores

Desea definir el presupuesto total de marketing (1: Si, 0: No): 0
Desea definir el costo máximo que se desea asumir por cliente (1: Si, 0: No): 0


### Restricción  capacidad call center

In [ ]:
definir=int(input('Desea definir la capacidad del call center, respecto a la cantidad de llamados por día (1: Si, 0: No): '))
if definir==1:
  Capacidad=int(input('Defina la capacidad del call center, respecto a la cantidad de llamados por día: ')) #80
elif definir==0:
  Capacidad=80
else:
  raise RuntimeError('Error: Solamente puede escribir 1 o 0, otros valores no son permitidos') # manejo de errores
Capacidad=Capacidad/2

Desea definir la capacidad del call center, respecto a la cantidad de llamados por día (1: Si, 0: No): 0


### Restricción del número máximo de interraciones en los últimos 3 meses.

In [ ]:
#Restricción por interacciones últimos 3 meses
definir=int(input('Desea definir el número máximo de interacciones permitidas en los últimos 3 meses (1: Si, 0: No): '))
if definir==1:
  Restriccion_canal=[]
  for i in range(0, len(Canales)): 
      interacciones = float(input('Defina el número máximo de interacciones permitidas en los últimos 3 meses '+Canales[i]+': ')) # 2, 8, 4
      Restriccion_canal.append(interacciones) # adding the element     
  Restriccion_canal=dict(zip(Canales,[Restriccion_canal[0], Restriccion_canal[1], Restriccion_canal[2]])) 

elif definir==0:
  Restriccion_canal={'CC':2, 'MAIL':8, 'SMS':4}
else:
  raise RuntimeError('Error: Solamente puede escribir 1 o 0, otros valores no son permitidos') # manejo de errores

Desea definir el número máximo de interacciones permitidas en los últimos 3 meses (1: Si, 0: No): 0


## Definir el conjunto de datos para los cuales pronosticar la probabilidad



En este paso se crea las tablas utilizadas para la aplicación del modelo generado de probabilidad de éxito y de rentabilidad. Las mismas representan datos de:

* Cadastro de clientes
* Rentabilidad
* Campañas realizadas

In [ ]:
#datos catastrales de clientes
Cadastro = []
for n in range(1,nro_clientes+1):
    Cadastro.append(list(faker.profile().values()))
Cadastro = pd.DataFrame(Cadastro, columns=faker.profile().keys())
Cadastro['ContaID'] = range(1,nro_clientes+1)

Cadastro_Aplic_Modelo = Cadastro[['ContaID', 'job', 'sex', 'birthdate']]

# edito variable job
for jobs in Mais_comuns.index:
  Cadastro_Aplic_Modelo['job_'+jobs]=np.where(Cadastro_Aplic_Modelo['job']==jobs,1,0)

Cadastro_Aplic_Modelo.loc[~Cadastro_Aplic_Modelo['job'].isin(Mais_comuns.index), 'job_Outros'] = 1
Cadastro_Aplic_Modelo.loc[Cadastro_Aplic_Modelo['job'].isin(Mais_comuns.index), 'job_Outros'] = 0

# edito variable job
Cadastro_Aplic_Modelo = pd.get_dummies(Cadastro_Aplic_Modelo, columns = ['sex'])

# edito variable Idade
ref_date = dt.datetime.now()
Cadastro_Aplic_Modelo['age'] = Cadastro_Aplic_Modelo['birthdate'].apply(lambda x: len(pd.date_range(start = x, end = ref_date, freq = 'Y'))) 

Cadastro_Aplic_Modelo = Cadastro_Aplic_Modelo.drop(['birthdate','job'], axis = 1)

## Creación del problema

### Creación de filtros

In [ ]:
Eleccion=[]
Rentabilidad_Producto = {}
Rentabilidad={}
Costos={}
Probabilidad={}

#elección del cliente
for id in range(1,nro_clientes+1):
  if id<10:
    cliente='_Cl000'+str(id)
  elif id<100:
    cliente='_Cl00'+str(id)
  elif id<1000:
    cliente='_Cl0'+str(id)
  else:
    cliente='_Cl'+str(id)

  Cadastro_Aplic_Modelo_Cliente=Cadastro_Aplic_Modelo[Cadastro_Aplic_Modelo['ContaID']==id]

  for Campanha in Productos:
    # predicción rentabilidad
    file_name='Rentabilidad_'+Productos_Res[Productos.index(Campanha)]+'.pkl'
    modelo_rentabilidad=pd.read_pickle('sample_data/'+file_name)

    Rentabilidad_Producto.update({Productos_Res[Productos.index(Campanha)]+
                                  cliente: list(modelo_rentabilidad.predict(Cadastro_Aplic_Modelo_Cliente.drop('ContaID', axis = 1)))[0]})

  for Campanha in Productos: 
    # predicción de la probabilidad
    for can in X2.columns[14:]:
      Cadastro_Aplic_Modelo_Cliente[can]=0

    file_name='Probabilidad_'+Productos_Res[Productos.index(Campanha)]+'.pkl'
    Prob=pd.read_pickle('sample_data/'+file_name)

    for canales in lista_canales:
      Cadastro_Aplic_Modelo_Cliente['Canal_'+canales]=1
      Probabilidad.update({canales+'_'+Productos_Res[Productos.index(Campanha)]+
                           cliente: list(Prob.predict_proba(Cadastro_Aplic_Modelo_Cliente.drop('ContaID', axis = 1))[:,1])[0]})
      Cadastro_Aplic_Modelo_Cliente['Canal_'+canales]=0
  


  #Cantidad de interacciones últimos 3 meses
  Usos_canal={'CC'+cliente: np.random.binomial(n=Restriccion_canal['CC'], p=0.4, size=None), 
              'MAIL'+cliente: np.random.binomial(n=Restriccion_canal['MAIL'], p=0.9, size=None), 
              'SMS'+cliente:np.random.binomial(n=Restriccion_canal['SMS'], p=0.7, size=None),
              'NONE'+cliente:0} 
  Usos_canal.update({'ALL'+cliente:Usos_canal['CC'+cliente]+Usos_canal['MAIL'+cliente]+Usos_canal['SMS'+cliente]})


  for i in range(len(El)):
    Eleccion.append(El[i]+cliente)

    if re.search('SA', El[i]):
        Rentabilidad.update({El[i]+cliente:Rentabilidad_Producto[Productos_Res[0]+cliente]})
    else:
        Rentabilidad.update({El[i]+cliente:Rentabilidad_Producto[Productos_Res[1]+cliente]})


  #Crea la lista de costos
  Costos.update({'ALL_SA'+cliente:Costos_canal['CC']+Costos_canal['MAIL']+Costos_canal['SMS'], 
                'CC_MAIL_SA'+cliente:Costos_canal['CC']+Costos_canal['MAIL'],
                'CC_SMS_SA'+cliente:Costos_canal['CC']+Costos_canal['SMS'], 
                'MAIL_SMS_SA'+cliente:Costos_canal['MAIL']+Costos_canal['SMS'], 
                'CC_SA'+cliente:Costos_canal['CC'],
                'MAIL_SA'+cliente:Costos_canal['MAIL'], 
                'SMS_SA'+cliente:Costos_canal['SMS'], 
                'NONE_SA'+cliente:0,

                'ALL_PR'+cliente:Costos_canal['CC']+Costos_canal['MAIL']+Costos_canal['SMS'], 
                'CC_MAIL_PR'+cliente:Costos_canal['CC']+Costos_canal['MAIL'],
                'CC_SMS_PR'+cliente:Costos_canal['CC']+Costos_canal['SMS'], 
                'MAIL_SMS_PR'+cliente:Costos_canal['MAIL']+Costos_canal['SMS'], 
                'CC_PR'+cliente:Costos_canal['CC'],
                'MAIL_PR'+cliente:Costos_canal['MAIL'], 
                'SMS_PR'+cliente:Costos_canal['SMS'], 
                'NONE_PR'+cliente:0})



  #creo diccionario de póliticas por canal
  contact_policy={}
  for j in Canales:
    for i in list(Probabilidad.keys()):
      match=re.search(j, i)
      if (match) and (Usos_canal[j+cliente]==Restriccion_canal[j]):
        contact_policy.update({i:0})
      elif (match) and (Usos_canal[j+cliente]!=Restriccion_canal[j]):
        contact_policy.update({i:np.random.binomial(n=1, p=0.95, size=None)}) #aca le doy un 5% de chances que el cliente diga que no quiere el producto


### instanciar problema y creación de variables

In [ ]:
# Creación del problema, el que va a tener los datos
prob = LpProblem("The_Marketing_Problem", LpMaximize)

# variables de activación de cada tensor
Eleccion_vars = LpVariable.dicts("E",Eleccion,0,1,LpInteger)

### Establecimiento de la función objetivo

En este paso definimos la función objetivo

In [ ]:
prob += lpSum([(Probabilidad[i]*Rentabilidad[i]-Costos[i])*Eleccion_vars[i] for i in Eleccion]), "Total Rentabilidad"
#print(prob)

### Restricciones a nivel cliente

En este paso lo que se establece es que para cada cliente haya una sola acción y por otro lado que el costo asociado a cada cliente no supere un cierto valor definido por el usuario. Además le pedimos que si el usuario manifestó que no quiere recibir determinado tipo de comunicación no la reciba (restricción de contact policy).

In [ ]:
for id in range(1,nro_clientes+1):
  if id<10:
    cliente='_Cl000'+str(id)
  elif id<100:
    cliente='_Cl00'+str(id)
  elif id<1000:
    cliente='_Cl0'+str(id)
  else:
    cliente='_Cl'+str(id)

  #me quedo con las variables inherentes al prestamo
  Eleccion_PR=[]
  for i in list(Probabilidad.keys()):
    match=re.search(r'PR'+cliente,i)
    if match:
      Eleccion_PR.append(i)

  #me quedo con las variables inherentes al Seguro Automovil
  Eleccion_SA=[]
  for i in list(Probabilidad.keys()):
    match=re.search(r'SA'+cliente, i)
    if match:
      Eleccion_SA.append(i)

  #me quedo con las variables inherentes al cliente
  Eleccion_Cl=[]
  for i in list(Probabilidad.keys()):
    match=re.search(cliente, i)
    if match:
      Eleccion_Cl.append(i)

# Añadimos las restricciones de la misma manera que la variable objetivo
  prob += lpSum([Eleccion_vars[i] for i in Eleccion_PR]) == 1, "EleccionFinal_PR %s" %  str(cliente) #haya alguna acción para el producto préstamo
  prob += lpSum([Eleccion_vars[i] for i in Eleccion_SA]) == 1, "EleccionFinal_SA %s" %  str(cliente) #haya alguna acción para el producto Seguro automovil
  prob += lpSum([Costos[i]*Eleccion_vars[i] for i in Eleccion_Cl]) <= Costo_cliente, "costo %s" %  str(cliente) #temas de costos por cliente

# restricciones de contact policy
for i in list(contact_policy.keys()):
  prob += Eleccion_vars[i]<= contact_policy[i] , "ContactPolicy %s" %  str(i)

### Restricciones generales
Aca es donde se aplican las restricciones de presupuesto de marketing y de capacidad del call center.

In [ ]:
#me quedo con las variables inherentes a call center
Eleccion_CC=[]
for i in list(Probabilidad.keys()):
  match_cc=re.search(r'CC', i)
  match_all=re.search(r'ALL', i)
  if (match_cc or match_all):
    Eleccion_CC.append(i)

# restricción de presupuesto de marketing
prob += lpSum([Costos[i]*Eleccion_vars[i] for i in Eleccion]) <= Costo_total, "Costo total" 

# restricción de capacidad
prob += lpSum([Eleccion_vars[i] for i in Eleccion_CC]) <= Capacidad, "Capacidad Call Center" 

### Ejecución del problema, status de la solución y acciones

Aqui se 

In [ ]:
#prob.writeLP("The_Marketing_Problem.lp")
prob.solve()

1

In [ ]:
print("Status:", LpStatus[prob.status])

Status: Optimal


### Acciones a realizar y evaluación de la función objetivo

In [ ]:
print('Acciones:')
for v in prob.variables():
  if (v.varValue==1):
  #Canal
    if re.search('NONE', v.name):
      V1='NO ofrecer'
    elif re.search('CC_ALL', v.name):
      V1='Ofrecer por TODOS los canales'
    elif re.search('CC_SMS', v.name):
      V1='Ofrecer por SMS y CALL CENTER'
    elif re.search('CC_MAIL', v.name):
      V1='Ofrecer por MAIL y CALL CENTER'
    elif re.search('MAIL_SMS', v.name):
      V1='Ofrecer por MAIL y SMS'
    elif re.search('CC', v.name):
      V1='Ofrecer por el CALL CENTER'
    elif re.search('SMS', v.name):
      V1='Ofrecer por SMS'
    elif re.search('MAIL', v.name):
      V1='Ofrecer por MAIL'
    else:
      V1='Ofrecer por TODOS los canales'
  #producto
    if re.search('SA', v.name):
      V2='Seguro de Automovil'
    else:
      V2='Prestamo'
    print(V1+' el producto '+V2+' al cliente '+v.name[-3:])

Acciones:
Ofrecer por el CALL CENTER el producto Seguro de Automovil al cliente 025
Ofrecer por el CALL CENTER el producto Seguro de Automovil al cliente 044
Ofrecer por el CALL CENTER el producto Seguro de Automovil al cliente 072
Ofrecer por SMS y CALL CENTER el producto Prestamo al cliente 012
Ofrecer por SMS y CALL CENTER el producto Prestamo al cliente 050
Ofrecer por SMS y CALL CENTER el producto Prestamo al cliente 069
Ofrecer por MAIL el producto Seguro de Automovil al cliente 039
Ofrecer por MAIL el producto Seguro de Automovil al cliente 051
Ofrecer por MAIL y SMS el producto Prestamo al cliente 001
Ofrecer por MAIL y SMS el producto Prestamo al cliente 002
Ofrecer por MAIL y SMS el producto Prestamo al cliente 003
Ofrecer por MAIL y SMS el producto Prestamo al cliente 004
Ofrecer por MAIL y SMS el producto Prestamo al cliente 005
Ofrecer por MAIL y SMS el producto Prestamo al cliente 006
Ofrecer por MAIL y SMS el producto Prestamo al cliente 007
Ofrecer por MAIL y SMS el pro

In [ ]:
print("Función de valor: ", int(value(prob.objective)))

Función de valor:  1217
